In [ ]:
!pip install datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import pandas as pd
import librosa
import numpy as np

import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

import datasets
from datasets import load_dataset

from datasets import DatasetDict, load_metric

In [4]:
from keras import layers,models,optimizers,losses
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

In [5]:
data = pd.read_csv("./drive/Shareddrives/SWDA/swb1_d1/text_data_nathan_with_timestamps.csv")
data = data.dropna()

In [6]:
data['file']="./drive/Shareddrives/SWDA/swb1_d1/data/sw0" + data['file'].str.replace(".txt",".sph")
data['difference'] = data['end']-data['start']
data[data.difference <= 0.0001]

,Channel,Utterance,Label,file,start,end,difference
43179,B,okay,"fo_o_fw_""_by_bc",./drive/Shareddrives/SWDA/swb1_d1/data/sw04927...,0.139625,0.139625,0.0


In [7]:
data = data.drop([43179])

In [8]:
data[data.difference <= 0.0001]

,Channel,Utterance,Label,file,start,end,difference


In [9]:
data_small=data[data.difference<=5]
data_small=data_small
data_small['Label'].value_counts()

sd                 10606
sv                  3299
b                   3120
%                   2287
aa                  1676
qy                   987
ba                   571
ny                   541
fo_o_fw_"_by_bc      516
qw                   507
qo                   269
nn                   268
bk                   254
h                    210
na                   194
qy^d                 185
ad                   179
fp                   178
bh                   167
^q                   150
b^m                  135
fc                   130
bf                   120
^h                   106
^2                   102
ng                    83
qh                    78
br                    70
ar                    69
no                    56
arp_nd                48
qrr                   40
oo_co_cc              30
bd                    27
qw^d                  19
t1                    17
aap_am                17
^g                    16
fa                    14
ft                     7


In [ ]:
labels_remove = data_small['Label'].value_counts().tail(27).index.to_list()
data_small = data_small.loc[~data_small.Label.isin(labels_remove)]

In [18]:
data_small['Label'].value_counts().size

41

In [ ]:
data_small.to_csv("./drive/Shareddrives/SWDA/swb1_d1/data_small_RNN.csv",index=False)

ds = load_dataset("csv",data_files='./drive/Shareddrives/SWDA/swb1_d1/data_small_RNN.csv',split='train')

In [ ]:
ds = ds.class_encode_column("Label")

In [13]:
def create_features(batch):

  y,sr = librosa.load(batch["file"], sr=16000, mono=True,offset=batch['start'],duration=batch['end']-batch['start'])
  arr = librosa.feature.mfcc(y=y,n_mfcc=128,sr=sr,hop_length=512 )
  print(arr.shape)
  # z = array.shape[2]
  h = arr.shape[0]
  w = arr.shape[1]

  a = (128 - h) // 2
  aa = 128 - a - h

  b = (200 - w) // 2
  bb = 200 - b - w
  batch['input'] = np.pad(arr, pad_width=((a, aa), (b, bb)), mode='constant')
  batch['label'] = batch['Label']
  return batch

In [ ]:
ds = ds.map(
        create_features,
        num_proc=4,
    )


In [15]:
ds = ds.train_test_split(test_size=0.1, stratify_by_column="Label",shuffle=True)
# INTRODUCE TRAIN TEST VAL SPLITS

# 90% train, 10% test + validation
train_valid = ds["train"].train_test_split(shuffle=True, test_size=0.1,stratify_by_column="Label")

# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_valid['train'],
    'test': ds['test'],
    'val': train_valid['test']})

In [16]:
train = ds['train'].to_tf_dataset(
          columns=["input"],
            label_cols=["Label"],
            batch_size=50,
)
test = ds['test'].to_tf_dataset(
          columns=["input"],
            label_cols=["Label"],
            batch_size=50,
)
val = ds['val'].to_tf_dataset(
          columns=["input"],
            label_cols=["Label"],
            batch_size=50,
)

In [17]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from keras.layers.attention.multi_head_attention import activation
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(256,input_shape=(128,200)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(41,activation='softmax'))


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=keras.losses.sparse_categorical_crossentropy,metrics =['acc'])
model.fit(train, epochs=10,
          validation_data=val, verbose = 1)

In [ ]:
train.save("train_MFCC")

In [ ]:
TrainLoss, Trainacc = model.evaluate(train)
TestLoss, Testacc = model.evaluate(test)
y_pred=model.predict(test)
print(Testacc)

51/51 [==============================] - 108s 2s/step
0.506369411945343
